In [1]:
import mongo_queries
from bson.objectid import ObjectId
import datetime
import utility_func

In [2]:
late = mongo_queries.late
leg = mongo_queries.leg
run = mongo_queries.run

In [32]:
doc = {
    'utc':datetime.datetime.utcnow(),
    'profiles':{
        '1':
            [
                {'segment_start':0,
                'segment_end':7,
                'poll_rate':15},
                {'segment_start':7,
                'segment_end':19,
                'poll_rate':10},
                {'segment_start':19,
                'segment_end':24,
                'poll_rate':30}
            ],
        '2':
            [
                {'segment_start':0,
                'segment_end':7,
                'poll_rate':60},
                {'segment_start':7,
                'segment_end':19,
                'poll_rate':30},
                {'segment_start':19,
                'segment_end':24,
                'poll_rate':60}
            ],
        '3':
            [
                {'segment_start':0,
                'segment_end':24,
                'poll_rate':60},
            ],            
        
            
    }
}

In [33]:
doc['profiles']['3']

[{'segment_start': 0, 'segment_end': 24, 'poll_rate': 60}]

In [5]:
db = mongo_queries.db

In [34]:
db['profiles'].insert_one(doc)
profs = db['profiles']

In [39]:
datetime.time(0,0)

datetime.time(0, 0)

In [40]:
profile = '1'
time = datetime.datetime(2021,11,18,12,34)

In [41]:
def get_poll_rate(profile, local_time):
    #returns the poll rate in number of minutes for the profile given local time
    #based on latest info from database
    #local_time has to be a datetime
    hour = local_time.hour
    #minu = local_time.minute #not needed?
    if type(profile) == int: #mongodb keys are strings
        profile  =str(profile)

    default = 60 # in case we can't find the rate
    latest = list(profs.aggregate([
        {'$sort':{'utc':-1}},
        {'$limit':1}
    ]))[0]
    try:
        sub_d = latest['profiles'][profile]
        for seg in sub_d:
            if hour in range(seg['segment_start'],seg['segment_end']):
                return seg['poll_rate']
        return default #if for whatever reason there was no time match
    except KeyError:
        return default
    

In [35]:
get_poll_rate(1,time)

NameError: name 'get_poll_rate' is not defined

In [43]:
time.mi

AttributeError: 'datetime.datetime' object has no attribute 'mi'

In [ ]:
10 in range(0,10)

False

In [24]:
sites = mongo_queries.get_updating_list()

In [25]:
sites

[427, 440, 602, 351, 453, 840]

In [9]:
import pandas as pd

In [10]:
my_list = [mongo_queries.get_last_run_base(i) for i in sites]

In [37]:

mongo_queries.get_poll_rate(1,datetime.datetime.utcnow())

30

In [38]:
datetime.datetime.utcnow()

datetime.datetime(2021, 11, 18, 22, 24, 4, 446731)

In [ ]:
df = pd.DataFrame(my_list)

In [ ]:
a = my_list[0]
a

{'id': 427,
 'name': 'Queenston Lewiston Bridge',
 'province': 'ON',
 'dest': '43.154644, -79.049258',
 'origin': '43.15273, -79.034308',
 'timeZone': 'America/Toronto',
 'region': 'Southern Ontario',
 'district': 'Niagara Falls',
 'profile': 1,
 'utc': datetime.datetime(2021, 11, 18, 21, 40),
 'wait': 154}

In [ ]:
url = utility_func.url_creator(a['origin'],a['dest'])
r = utility_func.get_data(url)
mongo_queries.add_one_base(r,a['id'])

In [ ]:
mongo_queries.get_last_run_base(427)

{'id': 427,
 'name': 'Queenston Lewiston Bridge',
 'province': 'ON',
 'dest': '43.154644, -79.049258',
 'origin': '43.15273, -79.034308',
 'timeZone': 'America/Toronto',
 'region': 'Southern Ontario',
 'district': 'Niagara Falls',
 'profile': 1,
 'utc': datetime.datetime(2021, 11, 18, 20, 50),
 'wait': 82}

In [11]:
my_list=[my_list[0]]

In [12]:
my_list

[{'id': 427,
  'name': 'Queenston Lewiston Bridge',
  'province': 'ON',
  'dest': '43.154644, -79.049258',
  'origin': '43.15273, -79.034308',
  'timeZone': 'America/Toronto',
  'region': 'Southern Ontario',
  'district': 'Niagara Falls',
  'profile': 1,
  'utc': datetime.datetime(2021, 11, 18, 21, 40),
  'wait': 154}]

In [14]:
import pytz

In [40]:
sites = [sites[0]]
for s in sites:
        x=mongo_queries.get_last_run_base(s)        
        id = x['id']                                                            #the id of the location
        utc = x['utc']
        utc = pytz.utc.localize(utc)                                            #the utc time it was last run
        tz = x['timeZone']                                                      #the time zone of the location
        ltnow = mongo_queries.convert_to_local(datetime.datetime.utcnow(),tz)   #local time now        
        lt = mongo_queries.convert_to_local(utc,tz)                             #local time at last time it was run
        poll_rate = mongo_queries.get_poll_rate(x['profile'],ltnow)             #how often it should be polled, in minutes
        poll_rate = datetime.timedelta(minutes=poll_rate)                       #convert the number of minutes into minutes (timedelta)
        now = datetime.datetime.utcnow()                                        #current utc time
        now = pytz.utc.localize(now)                                            #converted to aware
        time_since = now - utc                                                  #time since it was last polled
        due = time_since > poll_rate                                            #is it due to be polled
        if due:        
            url = utility_func.url_creator(x['origin'],x['dest'])               #create the url
            r = utility_func.get_data(url)                                      #send the request to google maps API
            mongo_queries.add_one_base(r,x['id'])                               #add it to the database. Note: DB reset --> now adds the data to the baseline collection as a sub array
        x.update({'local_time':lt,'time_since':time_since,'poll_rate':poll_rate,'due':due})  # dict of what we did, not needed other than a df?
        print(now,'- ', x['name'],' last run:', utc,' time since:', time_since,' due:',due)
        #print(x)

2021-11-18 22:24:55.641764+00:00 -  Queenston Lewiston Bridge  last run: 2021-11-18 22:20:00+00:00  time since: 0:04:55.641764  due: False


In [14]:
my_list

{'id': 427,
 'name': 'Queenston Lewiston Bridge',
 'province': 'ON',
 'dest': '43.154644, -79.049258',
 'origin': '43.15273, -79.034308',
 'timeZone': 'America/Toronto',
 'region': 'Southern Ontario',
 'district': 'Niagara Falls',
 'profile': 1,
 'utc': datetime.datetime(2021, 11, 18, 20, 50),
 'wait': 82}

In [23]:
now = datetime.datetime.utcnow()                                        #current utc time
now = pytz.utc.localize(now) 
mongo_queries.convert_to_local(now,tz) 

datetime.datetime(2021, 11, 18, 17, 4, 11, 12049, tzinfo=<DstTzInfo 'America/Toronto' EST-1 day, 19:00:00 STD>)